### Seção 6: Análise Estatística
Aplique testes estatísticos (como ANOVA ou teste t) para determinar se as diferenças de desempenho observadas entre as versões são estatisticamente significativas. Formule as hipóteses nula ($H_0$) e alternativa ($H_1$) e interprete os resultados do p-valor.

In [5]:
# Diretório para salvar os resultados finais
OUTPUT_DIR = BASE_DIR / 'results_archive' / 'v2.0.0-SNAPSHOT'
PLOTS_DIR = OUTPUT_DIR / 'plots'
LATEX_DIR = OUTPUT_DIR / 'latex'

# Cria os diretórios se não existirem
PLOTS_DIR.mkdir(parents=True, exist_ok=True)
LATEX_DIR.mkdir(parents=True, exist_ok=True)

# --- Exemplo 1: Salvar um gráfico de boxplot em alta qualidade ---
scenario_to_save = 'indisponibilidade-extrema'
fig, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(
    ax=ax,
    data=results_df[results_df['scenario'] == scenario_to_save],
    x='treatment',
    y='p95_duration_ms',
    order=[f'v{i}' for i in range(1, 5)]
)

ax.set_title(f'Distribuição do Tempo de Resposta (p95) - {scenario_to_save.replace("-", " ").title()}', fontsize=16)
ax.set_ylabel('Tempo de Resposta p95 (ms)', fontsize=12)
ax.set_xlabel('Versão do Serviço', fontsize=12)
ax.set_yscale('log')

# Salva em diferentes formatos
plot_filename_base = f'boxplot_p95_{scenario_to_save}'
fig.savefig(PLOTS_DIR / f'{plot_filename_base}.pdf', bbox_inches='tight') # Vetorial, ideal para LaTeX
fig.savefig(PLOTS_DIR / f'{plot_filename_base}.png', dpi=300, bbox_inches='tight') # Alta resolução

print(f"Gráfico salvo em: {PLOTS_DIR / f'{plot_filename_base}.pdf'}")
plt.show()


# --- Exemplo 2: Exportar um DataFrame para uma tabela LaTeX ---
# Vamos exportar a tabela de análise agrupada
latex_table = grouped_analysis.to_latex(
    index=False,
    float_format="%.2f", # Formata floats com 2 casas decimais
    caption='Análise Agrupada das Métricas de Performance por Cenário e Versão.',
    label='tab:grouped_analysis',
    column_format='llrrrrrr' # Define o alinhamento das colunas
)

latex_filename = LATEX_DIR / 'tabela_resumo_agrupado.tex'
with open(latex_filename, 'w') as f:
    f.write(latex_table)

print(f"\nTabela LaTeX salva em: {latex_filename}")
# Exibe o código LaTeX gerado
# print("\nCódigo LaTeX gerado:")
# print(latex_table)


NameError: name 'BASE_DIR' is not defined

### Seção 8: Geração de Gráficos para Publicação (LaTeX)
Produza gráficos de alta qualidade com `matplotlib` e `seaborn`, com rótulos claros, títulos e legendas. Demonstre como salvar esses gráficos em formatos vetoriais (como PDF ou SVG) ou de alta resolução (PNG) para inclusão em documentos LaTeX.

In [ ]:
def load_full_run_data(experiment_dir: Path, scenario: str, treatment: str, run: int) -> pd.DataFrame:
    """Carrega os dados de série temporal de um arquivo de execução completo."""
    filepath = experiment_dir / f"{scenario}_{treatment}_run{run}.json"
    
    if not filepath.exists():
        print(f"Arquivo não encontrado: {filepath}")
        return pd.DataFrame()

    points = []
    with open(filepath, 'r') as f:
        for line in f:
            try:
                record = json.loads(line)
                # Filtra apenas os pontos de dados de métricas de série temporal
                if record['type'] == 'Point' and 'data' in record:
                    # Converte o timestamp para datetime
                    timestamp = pd.to_datetime(record['data']['time'])
                    metric_name = record['metric']
                    value = record['data']['value']
                    points.append([timestamp, metric_name, value])
            except (json.JSONDecodeError, KeyError):
                continue # Ignora linhas que não são JSON ou não têm a estrutura esperada
    
    if not points:
        return pd.DataFrame()

    df = pd.DataFrame(points, columns=['time', 'metric', 'value'])
    df = df.set_index('time')
    return df

# Exemplo: Analisar a série temporal da duração das requisições para uma execução
# Vamos escolher um cenário e uma execução para demonstrar
scenario_to_plot = 'falha-catastrofica'
treatment_to_plot = 'v4' # Ex: A versão com Circuit Breaker
run_to_plot = 1

print(f"Carregando dados detalhados para: {scenario_to_plot}, {treatment_to_plot}, run {run_to_plot}")
full_run_df = load_full_run_data(EXPERIMENT_DIR, scenario_to_plot, treatment_to_plot, run_to_plot)

if not full_run_df.empty:
    # Filtra a métrica de duração da requisição
    duration_ts = full_run_df[full_run_df['metric'] == 'http_req_duration']

    plt.figure(figsize=(15, 7))
    duration_ts['value'].plot(style='.', alpha=0.5)
    
    # Adiciona uma média móvel para visualizar a tendência
    duration_ts['value'].rolling('10s').mean().plot(color='red', linewidth=2)

    plt.title(f'Duração das Requisições ao Longo do Tempo\n({scenario_to_plot} - {treatment_to_plot} - Run {run_to_plot})')
    plt.ylabel('Duração (ms)')
    plt.xlabel('Tempo de Execução do Teste')
    plt.legend(['Duração da Requisição', 'Média Móvel (10s)'])
    plt.show()
else:
    print("Não foi possível gerar o gráfico de série temporal.")


### Seção 7: Análise de Séries Temporais das Métricas
Carregue os dados detalhados (arquivos `.json` sem `_summary`) para uma execução específica. Plote métricas como requisições por segundo (`http_reqs`) e latência (`http_req_duration`) ao longo do tempo para analisar o comportamento do sistema durante o teste.

In [ ]:
# Usaremos o teste de Mann-Whitney U, que não assume uma distribuição normal dos dados,
# o que é uma escolha segura para métricas de performance.

# Hipóteses:
# H0 (Hipótese Nula): As distribuições das métricas de duas versões são iguais.
# H1 (Hipótese Alternativa): As distribuições são diferentes.
# Se p-valor < 0.05, rejeitamos H0 e concluímos que há uma diferença estatisticamente significativa.

from itertools import combinations

statistical_results = []

# Métricas para testar
metrics_to_test = ['success_rate', 'p95_duration_ms']

for scenario in results_df['scenario'].unique():
    for metric in metrics_to_test:
        # Gera todas as combinações de pares de tratamentos (v1 vs v2, v1 vs v3, etc.)
        treatment_pairs = list(combinations(results_df['treatment'].unique(), 2))
        
        for v_a, v_b in treatment_pairs:
            # Seleciona os dados para cada versão no cenário atual
            data_a = results_df[(results_df['scenario'] == scenario) & (results_df['treatment'] == v_a)][metric]
            data_b = results_df[(results_df['scenario'] == scenario) & (results_df['treatment'] == v_b)][metric]
            
            # Realiza o teste
            if len(data_a) > 1 and len(data_b) > 1:
                stat, p_value = stats.mannwhitneyu(data_a, data_b, alternative='two-sided')
                
                statistical_results.append({
                    'scenario': scenario,
                    'metric': metric,
                    'comparison': f'{v_a} vs {v_b}',
                    'statistic': stat,
                    'p_value': p_value,
                    'significant_at_5%': 'Sim' if p_value < 0.05 else 'Não'
                })

# Cria um DataFrame com os resultados estatísticos
stats_df = pd.DataFrame(statistical_results)

print("Resultados da Análise Estatística (Mann-Whitney U):")
with pd.option_context('display.max_rows', None): # Mostra todas as linhas
    display(stats_df)


In [ ]:
# Gráfico de Barras: Taxa de Sucesso Média por Cenário
for scenario in results_df['scenario'].unique():
    plt.figure(figsize=(10, 5))
    
    sns.barplot(
        data=grouped_analysis[grouped_analysis['scenario'] == scenario],
        x='treatment',
        y='avg_success_rate',
        order=[f'v{i}' for i in range(1, 5)] # Garante a ordem v1, v2, v3, v4
    )
    
    plt.title(f'Taxa de Sucesso Média - Cenário: {scenario.replace("-", " ").title()}')
    plt.ylabel('Taxa de Sucesso Média')
    plt.xlabel('Versão do Serviço')
    plt.ylim(0, 1.05) # Limite do eixo y para taxa (0 a 1)
    
    # Salvar o gráfico
    plot_filename = PLOTS_DIR / f'bar_avg_success_rate_{scenario}.png'
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_filename}")
    
    plt.show()

# Box Plot: Distribuição do Tempo de Resposta (p95) por Cenário
for scenario in results_df['scenario'].unique():
    plt.figure(figsize=(12, 7))
    
    sns.boxplot(
        data=results_df[results_df['scenario'] == scenario],
        x='treatment',
        y='p95_duration_ms',
        order=[f'v{i}' for i in range(1, 5)]
    )
    
    plt.title(f'Distribuição do Tempo de Resposta (p95) - Cenário: {scenario.replace("-", " ").title()}')
    plt.ylabel('Tempo de Resposta p95 (ms)')
    plt.xlabel('Versão do Serviço')
    plt.yscale('log') # Usar escala logarítmica pode ajudar a visualizar melhor grandes variações
    
    # Salvar o gráfico
    plot_filename = PLOTS_DIR / f'boxplot_p95_duration_{scenario}.png'
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo em: {plot_filename}")
    
    plt.show()


### Seção 5: Visualização de Métricas de Desempenho
Crie visualizações para comparar o desempenho entre as diferentes versões em cada cenário. Use gráficos de barras para comparar médias e box plots para visualizar a distribuição dos tempos de resposta (ex: `http_req_duration`).

In [ ]:
# Agrupa por cenário e tratamento para análise
grouped_analysis = results_df.groupby(['scenario', 'treatment']).agg(
    avg_success_rate=('success_rate', 'mean'),
    std_success_rate=('success_rate', 'std'),
    avg_p95_duration=('p95_duration_ms', 'mean'),
    std_p95_duration=('p95_duration_ms', 'std'),
    avg_duration=('avg_duration_ms', 'mean'),
    std_duration=('avg_duration_ms', 'std'),
    run_count=('run', 'count')
).reset_index()

print("Análise Agrupada por Cenário e Versão:")
display(grouped_analysis)


### Seção 4: Análise Comparativa por Cenário e Versão
Agrupe os dados por cenário e versão. Calcule a média, desvio padrão e outras estatísticas agregadas para as principais métricas de desempenho, como tempo de resposta (média, p(95)) e taxa de falha.

In [ ]:
# Exibe as primeiras linhas para verificação
print("Dados carregados:")
display(results_df.head())

# Informações gerais sobre o DataFrame
print("\nInformações do DataFrame:")
results_df.info()

# Estatísticas descritivas
print("\nEstatísticas Descritivas:")
display(results_df.describe().T)

# Verifica dados ausentes
print("\nVerificação de Dados Ausentes:")
print(results_df.isnull().sum())


### Seção 3: Análise Exploratória dos Dados (EDA)
Exiba as primeiras linhas do DataFrame para verificar o carregamento. Use `.describe()` para obter estatísticas descritivas das métricas numéricas. Verifique se há dados ausentes e realize a limpeza de dados, se necessário.

In [ ]:
# ATENÇÃO: Ajuste o caminho base se o seu notebook não estiver na raiz do projeto.
# Como estamos em analysis/analysis_v2.ipynb, subimos um nível para a raiz do projeto.
# O WSL usa caminhos no estilo Linux, então o Path funciona bem.
BASE_DIR = Path(os.getcwd()).parent
EXPERIMENT_DIR = BASE_DIR / 'k6' / 'results' / 'comparative' / 'experiment_20251216_101442'

def load_summary_files(experiment_dir: Path) -> pd.DataFrame:
    """Carrega todos os arquivos *_summary.json do diretório."""
    records = []
    
    for f in experiment_dir.glob("*_summary.json"):
        filename = f.stem.replace("_summary", "")
        # Formato: cenario_treatment_runN (cenario pode conter hífens)
        match = re.match(r"^(.+)_(v\d+)_(run\d+)$", filename)
        
        if match:
            scenario = match.group(1)
            treatment = match.group(2)
            run = int(match.group(3).replace("run", ""))
            
            with open(f) as fp:
                data = json.load(fp)
            
            metrics = data.get("metrics", {})
            
            # Extrai as métricas de interesse
            reqs_metric = metrics.get("http_reqs", {})
            failed_metric = metrics.get("http_req_failed", {})
            duration_metric = metrics.get("http_req_duration", {})
            iterations_metric = metrics.get("iterations", {})

            record = {
                "scenario": scenario,
                "treatment": treatment,
                "run": run,
                "total_requests": reqs_metric.get("count", 0),
                "fail_rate": failed_metric.get("rate", 0),
                "avg_duration_ms": duration_metric.get("avg", 0),
                "p95_duration_ms": duration_metric.get("p(95)", 0),
                "p99_duration_ms": duration_metric.get("p(99)", 0),
                "iterations": iterations_metric.get("count", 0),
            }
            records.append(record)
    
    df = pd.DataFrame(records)
    # Adiciona a taxa de sucesso para facilitar a análise
    if not df.empty:
        df['success_rate'] = 1 - df['fail_rate']
        
    return df

# Carrega os dados
results_df = load_summary_files(EXPERIMENT_DIR)


### Seção 2: Carregamento e Pré-processamento dos Dados
Defina o caminho para a pasta de resultados, considerando o ambiente WSL. Crie uma função para percorrer os diretórios, ler os arquivos JSON de sumário (`_summary.json`), extrair métricas chave (como `http_req_duration`, `http_req_failed`, `iterations`) e informações dos nomes dos arquivos (cenário, versão, execução). Consolide tudo em um único DataFrame do Pandas.

In [ ]:
import os
import re
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px

# Configurações de visualização
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


### Seção 1: Configuração do Ambiente e Importação de Bibliotecas
Importe as bibliotecas necessárias para a análise, como pandas, os, json, matplotlib e seaborn. Defina as configurações iniciais para os gráficos.

# Análise Comparativa de Performance - Versão 2.0.0

Este notebook realiza a análise dos dados de performance coletados via k6. O processo inclui carregamento de dados, análise estatística, visualização e exportação de resultados.

**Diretório de dados**: `k6/results/comparative/experiment_20251216_101442`

### Seção 9: Conclusão

Resuma os principais achados da análise.
Destaque quais versões tiveram melhor desempenho em quais cenários, com base nas evidências estatísticas e visuais.
Aponte observações importantes (ex: onde o Circuit Breaker foi mais eficaz).